<a href="https://colab.research.google.com/github/soulvi/ML/blob/main/mlops_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

requirements.txt:

kaggle
pandas
numpy
scikit-learn

In [ ]:
#установка requirements в колабе
!pip install kaggle
!pip install pandas
!pip install numpy
!pip install scikit-learn

data_creation.py

In [ ]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split

data = pd.read_csv("cleaned_car.csv", delimiter = ',', index_col=0)

# Смотрим на типы данных
cat_columns = []
num_columns = []

for column_name in data.columns:
    if (data[column_name].dtypes == object):
        cat_columns +=[column_name]
    else:
        num_columns +=[column_name]

print('Категориальные данные:\t ',cat_columns, '\n Число столблцов = ',len(cat_columns))

print('Числовые данные:\t ',  num_columns, '\n Число столблцов = ',len(num_columns))

# Перекодируем значения в колонке Fuel_type
data.fuel_type=data.fuel_type.replace({'Petrol':0,'Diesel':1,'LPG':2})

# Приводим категориальные признаки к численным
df_se = data.copy()
df_se[cat_columns] = df_se[cat_columns].astype('category')
for _, column_name in enumerate(cat_columns):
    df_se[column_name] = df_se[column_name].cat.codes

# Определяем X и y, нам нужно предсказать цену
y = df_se["Price"]
X = df_se.drop(["Price"], axis = 1)

# Делим данные на тестовую и тренировочную выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Сохраняем тренировочные и тестовые выборки в CSV файлы
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

# Создаём папку для сохранения данных
os.makedirs("dataset/train", exist_ok=True)
os.makedirs("dataset/test", exist_ok=True)

train_data.to_csv('dataset/train/train_data.csv', index=False)
test_data.to_csv('dataset/test/test_data.csv', index=False)

Категориальные данные:	  ['name', 'company', 'fuel_type'] 
 Число столблцов =  3
Числовые данные:	  ['year', 'Price', 'kms_driven'] 
 Число столблцов =  3


model_preprocessing.py

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Загружаем данные для предобработки
train_data = pd.read_csv('dataset/train/train_data.csv')
test_data = pd.read_csv('dataset/test/test_data.csv')

# Разделяем предикторы и целевую переменную
X_train = train_data.drop(["Price"], axis=1)
y_train = train_data["Price"]
X_test = test_data.drop(["Price"], axis=1)
y_test = test_data["Price"]

# Производим стандартизацию данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Сохраняем предобработанные данные
train_data_scaled = pd.concat([pd.DataFrame(X_train_scaled), y_train], axis=1)
test_data_scaled = pd.concat([pd.DataFrame(X_test_scaled), y_test], axis=1)

train_data_scaled.to_csv('dataset/train/train_data_scaled.csv', index=False)
test_data_scaled.to_csv('dataset/test/test_data_scaled.csv', index=False)

model_preparation.py

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor  # Пример модели
import joblib  # Для сохранения модели

# Загрузка данных для обучения
train_data = pd.read_csv('dataset/train/train_data_scaled.csv')

# Разделение предикторов и целевой переменной
X_train = train_data.drop(["Price"], axis=1)
y_train = train_data["Price"]

# Создание и обучение модели
model = RandomForestRegressor()  # Пример модели
model.fit(X_train, y_train)

# Сохранение модели
joblib.dump(model, 'trained_model.pkl')

['trained_model.pkl']

model_testing.py

In [ ]:
import pandas as pd
from sklearn.metrics import mean_squared_error
import joblib
import json

# Загрузка данных для тестирования
test_data = pd.read_csv('dataset/test/test_data_scaled.csv')

# Разделение предикторов и целевой переменной
X_test = test_data.drop(["Price"], axis=1)
y_test = test_data["Price"]

# Загрузка обученной модели
model = joblib.load('trained_model.pkl')

# Предсказание на тестовых данных
predictions = model.predict(X_test)

# Оценка качества модели
mse = mean_squared_error(y_test, predictions)
print(f"Среднеквадратичная ошибка в тестовых данных: {mse}")

with open("testing_mse.json", "w", encoding="utf-8") as json_file:
    json.dump(mse, json_file, ensure_ascii=False)


Среднеквадратичная ошибка в тестовых данных: 58421935763.27224


pipeline.sh

In [ ]:
#!/bin/bash
echo "Устанавливаем переменные окружения Kaggle"
export KAGGLE_USERNAME="levshlyogin"
export KAGGLE_KEY="a34c45dc721891d278828abf7850aac4"

echo "Устанавливаем библиотеки и зависимости"
pip install -r requirements.txt

echo "Скачиваем датасет"
kaggle datasets download -d shivamkushwah0411/quicker-car-cleaned-dataset

echo "Создаём папку для сохранения данных"
mkdir -p dataset

echo "Извлекаем файл датасета в папку dataset"
unzip -o quicker-car-cleaned-dataset.zip -d dataset/quicker-car-cleaned-dataset

echo "Запускаем Python-скрипт для получения данных"
python3 data_creation.py
echo "data_creation.py Done!"
sleep 5

echo "Выполняем предобработку данных"
python3 model_preparation.py
echo "model_preparation.py Done!"
sleep 5

echo "Создаем и обучаем модель машинного обучения на построенных данных из папки train"
python3 model_preprocessing.py
echo "model_preprocessing.py Done!"
sleep 5

echo "Проверяем модель машинного обучения на построенных данных из папки test"
python3 model_testing.py
echo "model_testing.py Done!"
sleep 5